Visualizing and Predicting Trends for the Summer Olympics (Data from 1896-2008) 
------

##### Data:
The data for all the medalists from Summer Olympics from 1896-2008 for this project comes from [this article from the Guardian UK](http://www.theguardian.com/sport/datablog/2012/jun/25/olympic-medal-winner-list-data#data). The raw data can be found in the 'Download Data' section of the article. The raw data comes as a Google Spreadsheet, which cannot be scraped using xpath and xml, so we had to manually publish each sheet to get the html version. 

GDP and Population data comes from the [Quandl Database](https://www.quandl.com/). We call the API, which is data from WorldBank, to get gdp and population data for each country we are looking at. The USSR data comes also comes from another Quandl API, although this data is much less reliable, given the history of the USSR, it is difficult to find open data about their GDP and population. 

The latitude and longitude data comes from a Google developer source. ALL LINKS TO DATA PROVIDED BELOW

##### Goals:
The first goal of this project is to visualize the trends of which countries win medals from the Summer Olympics, from 1896-2008 (this is what is available). We want to see if we can explain any of these trends with the GDP and Population data. 

The second goal of this project is to predict how many medals will be won by the most historically dominant countries - U.S., U.K., Russia, and China. 

##### Method:
1. Get the data
2. Clean up the data frames, and customize them to be useable when building graphs and tables
3. Create the visualizations
4. Create the predictions

### Links to all data
Summer Olympics Data:
* [All medalists](https://docs.google.com/spreadsheets/d/138NErLS90hqFbxJN9PfdLory495Zeuld9336n1Ld1xQ/pubhtml?gid=0&single=true)
* [Breakdown of wins by country and gender](https://docs.google.com/spreadsheets/d/1CDMFU-0XGnJIGgsaq_Vhi8O3Jbqw4sYkwlsUaFq8sI4/pubhtml?gid=1&single=true)
* [Country Totals](https://docs.google.com/spreadsheets/d/1CDMFU-0XGnJIGgsaq_Vhi8O3Jbqw4sYkwlsUaFq8sI4/pubhtml?gid=3&single=true)

GDP:
* [China GDP](https://www.quandl.com/api/v3/datasets/WWDI/CHN_NY_GDP_MKTP_CD.json?api_key=wy1bWm_Px92HwwHawCbg)
* [Russia GDP](https://www.quandl.com/api/v3/datasets/WWDI/RUS_NY_GDP_MKTP_CD.json?api_key=wy1bWm_Px92HwwHawCbg)
* [UK GDP](https://www.quandl.com/api/v3/datasets/WWDI/GBR_NY_GDP_MKTP_CD.json?api_key=wy1bWm_Px92HwwHawCbg)
* [USA GDP](https://www.quandl.com/api/v3/datasets/WWDI/USA_NY_GDP_MKTP_CD.json?api_key=wy1bWm_Px92HwwHawCbg)

Population:
* [USA Population](https://www.quandl.com/api/v3/datasets/WWDI/USA_SP_POP_TOTL.json?api_key=wy1bWm_Px92HwwHawCbg)
* [UK Population](https://www.quandl.com/api/v3/datasets/WWDI/GBR_SP_POP_TOTL.json?api_key=wy1bWm_Px92HwwHawCbg)
* [China Population](https://www.quandl.com/api/v3/datasets/WWDI/CHN_SP_POP_TOTL.json?api_key=wy1bWm_Px92HwwHawCbg)
* [Russia Population](https://www.quandl.com/api/v3/datasets/WWDI/RUS_SP_POP_TOTL.json?api_key=wy1bWm_Px92HwwHawCbg)

Latitude and Longitude Data
* [Lat/Long](https://developers.google.com/public-data/docs/canonical/countries_csv)

## Overall Observations:
* Locations of Olympics are clustered around Western Europe, North America, understandeably so given climate, but also Australia - so it's clear that Olympics like to be held in countries that are rich and have strong infrastructure. This makes it interesting that Olympics this year is being held in Rio.
* Similarly, the countries with the highest medal count over time are mostly Western countries, with exception of Soviet Union - this suggests that countries with GDP and Population do well, but also that culture and politics are just as important.
* History and political climate - WWI, WWII, Cold War, etc. - is a major factor in Olympic Performance (visually looking at the graphs at least).
* GDP and Population can influence Olympic performance with some lag in time - for example the UK experienced a lag in GDP and Population in the 1970s-80s, which led to a drop in medals in the 1990s-2000s. 

## 2016 Medal Predictions:
- US: 363 Medals
- UK: 71 Medals
- China: 212 Medals
- Russia: 303 Medals
- DISCLAIMER: the medal count of the data counts every single medal given out - ex. if the U.S. Basketball wins gold, the data counted the medal for each player on the team. Thus U.S. has so many medals because they win a lot of team events.

### Step 1: Crawl the URLs and Use API for the Data and Put into Dataframes

In [ ]:
import requests
from lxml import html
import pandas as pd
import json

In [ ]:
#this function uses requests to grab the html from the url, uses xpath to target the tables and rows,
#and then adds the specific rows to the list

def crawl(url, x_path, skip):
    response = requests.get(url, verify=False) #verify is required because it's from google docs
    doc = html.fromstring(response.text)
    tables = doc.xpath(x_path) #use xpath to grab the tables and the rows
    table = tables[0]
    rows = table.xpath(".//tr")
    data = []
    for row_num in range(len(rows)): #add the rows to a list
        if row_num < skip:
            pass
        elif row_num > skip-1:
            cell = rows[row_num].xpath(".//td")
            data.append([val.text_content() for val in cell])
    return data

In [ ]:
#use this function if you are getting data by calling the Quandl API

def quandl_api(url):
    req = requests.get(url)
    data = json.loads(req.text)
    cells = data["dataset"]["data"] 
    columns = data["dataset"]["column_names"]
    return cells, columns
    

In [ ]:
#use function to grab data for 'Breakdown of Wins' and put them into a dataframe
wins = crawl("https://docs.google.com/spreadsheets/d/1CDMFU-0XGnJIGgsaq_Vhi8O3Jbqw4sYkwlsUaFq8sI4/pubhtml?gid=1&single=true", 
             ".//tbody", 3)
breakdown = pd.DataFrame(wins)

In [ ]:
#use function to grab data for 'Country Totals' and put them into a dataframe
country_wins = crawl("https://docs.google.com/spreadsheets/d/1CDMFU-0XGnJIGgsaq_Vhi8O3Jbqw4sYkwlsUaFq8sI4/pubhtml?gid=3&single=true",
                    ".//tbody", 3)
country_totals = pd.DataFrame(country_wins)

In [ ]:
#use function to grab data for 'All Medalists' and put them into a dataframe
all_medals = crawl("https://docs.google.com/spreadsheets/d/138NErLS90hqFbxJN9PfdLory495Zeuld9336n1Ld1xQ/pubhtml?gid=0&single=true",
             ".//tbody", 5)
header = ["City", "Year", "Sport", "Discipline", "Athlete", "Country", "Gender", "Event", "Event_Gender", "Medal"]
medalists = pd.DataFrame(all_medals, columns=header)

In [ ]:
#use the function quandl_api to grab gdp data and put into dataframe
ch_gdp, header = quandl_api("https://www.quandl.com/api/v3/datasets/WWDI/CHN_NY_GDP_MKTP_CD.json?api_key=wy1bWm_Px92HwwHawCbg")
ch_gdp = pd.DataFrame(ch_gdp, columns=header)

rs_gdp, header = quandl_api("https://www.quandl.com/api/v3/datasets/WWDI/RUS_NY_GDP_MKTP_CD.json?api_key=wy1bWm_Px92HwwHawCbg")
rs_gdp = pd.DataFrame(rs_gdp, columns=header)

uk_gdp, header = quandl_api("https://www.quandl.com/api/v3/datasets/WWDI/GBR_NY_GDP_MKTP_CD.json?api_key=wy1bWm_Px92HwwHawCbg")
uk_gdp = pd.DataFrame(uk_gdp, columns=header)

us_gdp, header = quandl_api("https://www.quandl.com/api/v3/datasets/WWDI/USA_NY_GDP_MKTP_CD.json?api_key=wy1bWm_Px92HwwHawCbg")
us_gdp = pd.DataFrame(us_gdp, columns=header)

In [ ]:
#use the function quandl_api to grab population data and put into dataframe
ch_pop, header = quandl_api("https://www.quandl.com/api/v3/datasets/WWDI/CHN_SP_POP_TOTL.json?api_key=wy1bWm_Px92HwwHawCbg")
ch_pop = pd.DataFrame(ch_pop, columns=header)

rs_pop, header = quandl_api("https://www.quandl.com/api/v3/datasets/WWDI/RUS_SP_POP_TOTL.json?api_key=wy1bWm_Px92HwwHawCbg")
rs_pop = pd.DataFrame(rs_pop, columns=header)

uk_pop, header = quandl_api("https://www.quandl.com/api/v3/datasets/WWDI/GBR_SP_POP_TOTL.json?api_key=wy1bWm_Px92HwwHawCbg")
uk_pop = pd.DataFrame(uk_pop, columns=header)

us_pop, header = quandl_api("https://www.quandl.com/api/v3/datasets/WWDI/USA_SP_POP_TOTL.json?api_key=wy1bWm_Px92HwwHawCbg")
us_pop = pd.DataFrame(us_pop, columns=header)

In [ ]:
#use crawl function to get the data for latitude and longitude for the countries

location = crawl("https://developers.google.com/public-data/docs/canonical/countries_csv", ".//table", 1)
header = ["Country_ID", "Latitude", "Longitude", "Country"]
location = pd.DataFrame(location, columns=header)

### Step 2: Clean up the Dataframes
- Most of the dataframes have multiple charts and tables combined into one giant table so we go through and manually split up the specific rows and columns to make individual tables
- the GDP, population, and location data must be merged together to make useful tables that match each country by their gdp, population, and latitude/longitude
- some of the data types need to be changed in order to be plotted in the next step

##### After cleaning up the dataframes, we end up with 6 dataframes total:
* medalists: dataframe of all individual medalists winners, including the medal won, in which event, and in which country
* gender_totals: dataframe of all medals won by males and females by each summer olympics
* country_host: dataframe of the count of all the countries that have hosted the olympics
* country: dataframe of a break of all medals by country
* countryovertime: dataframe of the top 5 nations (UK, Russia, US, and China) and their medal count by year
* demo_table: dataframe of top 5 nations, with their medal count by year, and GDP and Population data

In [ ]:
#cleaning up the 'Breakdown of Wins' dataframe to gather breakdown of genders dataframe
list_rows = range(33)
list_rows2 = range(60, 96)
gender_totals = breakdown.drop(list_rows) #drop rows
gender_totals = gender_totals.drop(list_rows2) #drop rows
gender_totals = gender_totals.drop([11,12,13], axis=1) #drop columns
gender_totals.columns = ["Year", "Bronze_Male", "Bronze_Female", "Bronze_Total", "Gold_Male", "Gold_Female",
                        "Gold_Total", "Silver_Male", "Silver_Female", "Silver_Total", "Grand_Total"]
gender_totals = gender_totals.reset_index() #re-index the dataframe
gender_totals = gender_totals.drop(["index"], axis=1) #drop index column

In [ ]:
#cleaning up the 'Breakdown of Wins' dataframe to gather count of Olympic Countries dataframe
list_rows = range(21,96)
country_host = breakdown.drop(list_rows) #drop rows
country_host = country_host.drop([0,1]) #drop rows
country_host = country_host.drop([0,1,2,3,4,5,6,7,8,9,10,11], axis=1) #drop columns
country_host.columns = ["Country", "Count"]
country_host = country_host.reset_index() #re-index the dataframe
country_host = country_host.drop(["index"], axis=1) #drop index column
country_host = pd.merge(country_host, location, on="Country", how="inner") #merge the latitude and longitude data with the country_host dataframe
country_host["Count"] = pd.to_numeric(country_host["Count"]) #convert the count into integer type
country_host["Latitude"] = pd.to_numeric(country_host["Latitude"])
country_host["Longitude"] = pd.to_numeric(country_host["Longitude"])

In [ ]:
#cleaning up the 'Country Totals' dataframe to gather breakdown of medals by country
list_rows = range(144)
country = country_totals.drop(list_rows) #drop rows
country = country.drop([6,7,8,9,10,11,12,13,14,15,16,17], axis=1) #drop columns
country.columns = ["Country_Code", "Country", "Bronze_Medals", "Gold_Medals", "Silver_Medals", "Total_Medals"]
country = country.reset_index()
country = country.drop(["index"], axis=1)
country = pd.merge(country, location, on="Country", how="outer") #merge the latitude and longitude data with the country_host dataframe
country = country.set_value(2, 'Latitude', 51.220611) #manually inputing latitude and longitude for Soviet Union
country = country.set_value(2, 'Longitude', 51.363587)
country = country.set_value(19, 'Latitude', 48.138965) #manually inputting latitude and longitude for West Germany
country = country.set_value(19, 'Longitude', 11.580024)
country = country.set_value(15, 'Latitude', 51.16762) #manually inputting latitude and longitude for East Germany
country = country.set_value(15, 'Longitude', 10.45905)

In [ ]:
#converting datatypes of country dataframe
country["Total_Medals"] = pd.to_numeric(country["Total_Medals"]) 
country["Latitude"] = pd.to_numeric(country["Latitude"])
country["Longitude"] = pd.to_numeric(country["Longitude"])
country["Gold_Medals"] = pd.to_numeric(country["Gold_Medals"])
country["Bronze_Medals"] = pd.to_numeric(country["Bronze_Medals"])
country["Silver_Medals"] = pd.to_numeric(country["Silver_Medals"])

In [ ]:
#cleaning up the 'Country Totals' dataframe to gather breakdown of medals for top 5 nations year over year
list_rows = range(30,283)
countryovertime = country_totals.drop(list_rows) #drop rows
countryovertime = countryovertime.drop([0, 27,28,29])#drop header row and percent total rows
countryovertime = countryovertime.drop([0,1,2,3,4,11,12,13,14,15,16,17], axis=1) #drop columns
countryovertime.columns = ["Year", "UK_Total_Medals", "USSR_Total_Medals", "Russia_Total_Medals", "USA_Total_Medals", "China_Total_Medals"]
countryovertime = countryovertime.reset_index()
countryovertime = countryovertime.drop(["index"], axis=1) 
countryovertime["Year"] = pd.to_numeric(countryovertime["Year"])

In [ ]:
#sets the float format for all databases
pd.options.display.float_format = '{:,.2f}'.format 

In [ ]:
#combine all the gdp data into one dataframe
test = pd.merge(ch_gdp, us_gdp, on='Date', how='inner')
test2 = pd.merge(test, uk_gdp, on='Date', how='inner')
top_gdp = pd.merge(test2, rs_gdp, on='Date', how='outer') #have to use outer join because russia is not established until 1989
top_gdp.columns=["Date", "CHN_GDP", "US_GDP", "UK_GDP", "RUS_GDP"]

In [ ]:
#combine all the gdp data into one dataframe
test = pd.merge(ch_pop, us_pop, on='Date', how='inner')
test2 = pd.merge(test, uk_pop, on='Date', how='inner')
top_pop = pd.merge(test2, rs_pop, on='Date', how='outer') #have to use outer join because russia is not established until 1989
top_pop.columns=["Date", "CHN_Pop", "US_Pop", "UK_Pop", "RUS_Pop"]

In [ ]:
#cleaning top_gdp data and merging the tables with countryovertime table
top_gdp["Date"] = pd.to_datetime(top_gdp["Date"])
y_list = list(top_gdp["Date"])
year = []
for i in y_list:
    year.append(i.year) #create a list of only the years
top_gdp["Year"] = year #add the year to the table (the year will be merged on for the countryovertime table)

top_gdp_table = pd.merge(countryovertime, top_gdp, on="Year", how="inner")

In [ ]:
#cleaning top_pop data and merging the tables with table made above
top_pop["Date"] = pd.to_datetime(top_pop["Date"])
y_list = list(top_pop["Date"])
year = []
for i in y_list:
    year.append(i.year) #create a list of only the years
top_pop["Year"] = year #add the year to the table (the year will be merged on for the countryovertime table)

demo_table = pd.merge(top_gdp_table, top_pop, on="Year", how="inner")
demo_table = demo_table.fillna(0)
demo_table["USA_Total_Medals"] = pd.to_numeric(demo_table["USA_Total_Medals"])
demo_table["UK_Total_Medals"] = pd.to_numeric(demo_table["UK_Total_Medals"])
demo_table["Russia_Total_Medals"] = pd.to_numeric(demo_table["Russia_Total_Medals"])
demo_table["USSR_Total_Medals"] = pd.to_numeric(demo_table["USSR_Total_Medals"])
demo_table["China_Total_Medals"] = pd.to_numeric(demo_table["China_Total_Medals"])
demo_table = demo_table.drop(["Date_x", "Date_y"], axis=1)

### Step 3: Make Visuals of the Data

In [ ]:
#only needs to be run the first time
#this one must be run in the terminal, because it prompts you after
!sudo apt-get install python-mpltoolkits.basemap

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
%matplotlib inline

# This defines the size of the image below
plt.rcParams['figure.figsize'] = (20, 18)
plt.style.use('seaborn-poster')

##### This Code Creates Map of All Countries that have Hosted Summer Olympic Games

In [ ]:

# Create a map on which to draw. We're using a `Gall Stereographic Cylindrical` projection, and showing the whole world.
# resolution: the resolution of the included coastlines, lakes, and so on. The crude resolution is chosen.
# area_thresh: the threshold under what no coast line or lake will be drawn.
# lat_0, lon_0: latitude and longitude of map center.
themap = Basemap(projection='robin', resolution = 'h', area_thresh = 10000.0, lat_0=0, lon_0=0)
 
# Draw coastlines, borders of countries and the edges of the map.
themap.drawcoastlines()
themap.drawcountries()
themap.fillcontinents(color='lightgray')
themap.drawmapboundary()

# Create a color code based on number of times hosted
def get_marker_color(count):
    # Returns green for countries that have hosted more than 3 times, yellow for more than once, and red for once
    if count > 3.0:
        return ('go')
    elif count > 1.0:
        return ('yo')
    else:
        return ('ro')
    
# plotting the text label of countries
labels = list(country_host["Country"])
lons = list(country_host["Longitude"])
lats = list(country_host["Latitude"])
x,y = themap(lons, lats)
for label, xpt, ypt in zip(labels, x, y):
    plt.text(xpt+10000, ypt+5000, label)

#plotting the points based on count
min_marker_size = 5.5
for lon, lat, cnt in zip(list(country_host["Longitude"]), list(country_host["Latitude"]), list(country_host["Count"])):
    x,y = themap(lon, lat)
    msize = cnt * min_marker_size
    marker_string = get_marker_color(cnt)
    themap.plot(x, y, marker_string, markersize=msize)
    
plt.title("All Countries that have Hosted Summer Olympic Games from 1896-2008")

plt.show()

##### This code creates a map showing the total medals for the top 10 countries from 1896-2008

In [ ]:
# This defines the size of the image below
plt.rcParams['figure.figsize'] = (20, 18)
plt.style.use('seaborn-poster')

# Create a map on which to draw. We're using a `Gall Stereographic Cylindrical` projection, and showing the whole world.
# resolution: the resolution of the included coastlines, lakes, and so on. The crude resolution is chosen.
# area_thresh: the threshold under what no coast line or lake will be drawn.
# lat_0, lon_0: latitude and longitude of map center.
themap = Basemap(projection='robin', resolution = 'h', area_thresh = 10000.0, lat_0=0, lon_0=0)
 
# Draw coastlines, borders of countries and the edges of the map.
themap.drawcoastlines()
themap.drawcountries()
themap.fillcontinents(color='lightgray')
themap.drawmapboundary()

#plotting the points based on count
min_marker_size = 6
for lon, lat, cnt in zip(list(country["Longitude"][1:11]), list(country["Latitude"][1:11]), list(country["Total_Medals"][1:11])):
    x,y = themap(lon, lat)
    msize = cnt / 1000
    msize = msize * min_marker_size
    themap.plot(x, y, 'bo', markersize=msize)
    
# plotting the text label of countries
labels = list(country["Country"][1:11])
lons = list(country["Longitude"][1:11])
lats = list(country["Latitude"][1:11])
x,y = themap(lons, lats)
for label, xpt, ypt in zip(labels, x, y):
    plt.text(xpt+10000, ypt+5000, label)
    
plt.title("Top Ten Countries with Highest Total Medal Count from 1896-2008")

plt.show()

##### This code is a bar chart of total medals from 1896-2008

In [ ]:
#MEDALS PER COUTNRY PLOT
cum_won = pd.DataFrame({"Country": country["Country"][1:30], 
                          "Total Medals": country["Total_Medals"][1:30],})

cum_won['Total Medals'] = cum_won['Total Medals'].astype(float)

#Plotting the table
cum_won.plot(kind="bar", x="Country", y=['Total Medals'])

plt.title('Total Medals By Country from 1896-2008')
plt.xlabel('Country')
plt.ylabel('Medals')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

##### Gold Medals Awarded by Gender from 1896-2008

In [ ]:
genders = gender_totals[:26]
genders

#Computing totals over time
gold_male = genders["Gold_Male"]
gold_female = genders["Gold_Female"]


#Create the dataframe with the cumulative data
gender_gold = pd.DataFrame({"Year": genders["Year"], 
                          "Male": gold_male,
                          "Female": gold_female})

for i in range(len(gender_gold["Female"])):
    if gender_gold["Female"][i] == "":
        gender_gold["Female"][i] = "0"

gender_df = gender_gold.astype(float)

#Plotting the table
gender_df.plot(kind="line", x="Year", y=['Male', 'Female'])

plt.title('Gold Medals By Gender Over Time', )
plt.xlabel('Year')
plt.ylabel('Gold Medals')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

##### Total Medals for 5 Key Countries Over Time (UK, USSR, Russia, USA, China)

In [ ]:
#Create the dataframe with the cumulative data for selected countries
topnationsyear_df = pd.DataFrame({"Year": countryovertime["Year"], 
                          "UK": countryovertime["UK_Total_Medals"],
                          "USSR": countryovertime["USSR_Total_Medals"],
                          "Russia": countryovertime["Russia_Total_Medals"],
                          "USA": countryovertime["USA_Total_Medals"],
                          "China": countryovertime["China_Total_Medals"]})

topnationsyeardf = topnationsyear_df.astype(float)

#Plotting the table
topnationsyeardf.plot(kind="line", x="Year", y=['UK', 'USSR', 'Russia', 'USA', 'China'])

plt.title('Medals over Time of 5 Key Countries', )
plt.xlabel('Year')
plt.ylabel('Medals')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

##### Total Medals Over Time for 5 Powerful Countries 
This required a little more cleaning of the data to get different individual counts. We use US, UK, Japan, South Korea, Germany, and France, to show the rise of Asian countries and the decline of older European countries in the Olympics.

In [ ]:
#create a table grouped by Year and Country, to be used to graph medals over time
medals_ot = medalists.groupby(['Year', 'Country']).size()
m_table = pd.DataFrame(medals_ot)
m_table = m_table.reset_index()
m_table.columns = ["Year", "Country", "Medal_Count"]

In [ ]:
#separate the Top Nations, in addition with Japan and South Korea
USA_medal = m_table.loc[m_table['Country'] == 'USA']
USA_medal["Year"] = pd.to_numeric(USA_medal["Year"])

UK_medal = m_table.loc[m_table['Country'] == 'GBR']
UK_medal["Year"] = pd.to_numeric(UK_medal["Year"])

Japan_medal = m_table.loc[m_table['Country'] == 'JPN']
Japan_medal["Year"] = pd.to_numeric(Japan_medal["Year"])

SK_medal = m_table.loc[m_table['Country'] == 'KOR']
SK_medal["Year"] = pd.to_numeric(SK_medal["Year"])

GER_medal = m_table.loc[m_table['Country'] == 'GER']
GER_medal["Year"] = pd.to_numeric(GER_medal["Year"])

FR_medal = m_table.loc[m_table['Country'] == 'FRA']
FR_medal["Year"] = pd.to_numeric(FR_medal["Year"])

In [ ]:
#merge individual tables together
top_table1 = pd.merge(USA_medal, UK_medal, on="Year", how="outer")
top_table2 = pd.merge(top_table1, Japan_medal, on="Year", how="outer")
top_table3 = pd.merge(top_table2, SK_medal, on="Year", how="outer")
top_table4 = pd.merge(top_table3, GER_medal, on="Year", how="outer")
top_table = pd.merge(top_table4, FR_medal, on="Year", how="outer")
top_table = top_table.fillna(0)
top_table = top_table.sort(["Year"], ascending=True)
top_table = top_table.drop(["Country_x", "Country_y", "Country_x", "Country_y", "Country_x", "Country_y"], axis=1)
top_table.columns=["Year", "US_Count", "UK_Count", "JPN_Count", "SK_Count", "GER_Count", "FR_Count"]

In [ ]:
top_table.plot(kind = "line", x="Year", y=["US_Count", "UK_Count", "JPN_Count", "SK_Count", "GER_Count", "FR_Count"])
plt.title('Total Medals Over Time', )
plt.xlabel('Year')
plt.ylabel('Medals')

##### Total Medals for BRICS Countries Over Time
We want to look at the five main emerging countries and how they've done.

In [ ]:
#separate table for BRICS (Brazil, Russia, India, China, South Africa)
BR_medal = m_table.loc[m_table['Country'] == 'BRA']
BR_medal["Year"] = pd.to_numeric(BR_medal["Year"])

RS_medal = m_table.loc[m_table['Country'] == 'RUS']
RS_medal["Year"] = pd.to_numeric(RS_medal["Year"])

IN_medal = m_table.loc[m_table['Country'] == 'IND']
IN_medal["Year"] = pd.to_numeric(IN_medal["Year"])

CH_medal = m_table.loc[m_table['Country'] == 'CHN']
CH_medal["Year"] = pd.to_numeric(CH_medal["Year"])

SA_medal = m_table.loc[m_table['Country'] == 'RSA']
SA_medal["Year"] = pd.to_numeric(SA_medal["Year"])

In [ ]:
#merge individual tables together for BRICS
BRICS_table1 = pd.merge(BR_medal, RS_medal, on="Year", how="outer")
BRICS_table2 = pd.merge(BRICS_table1, IN_medal, on="Year", how="outer")
BRICS_table3 = pd.merge(BRICS_table2, CH_medal, on="Year", how="outer")
BRICS_table = pd.merge(BRICS_table3, SA_medal, on="Year", how="outer")
BRICS_table = BRICS_table.fillna(0)
BRICS_table = BRICS_table.sort(["Year"], ascending=True)
BRICS_table = BRICS_table.drop(["Country_x", "Country_y", "Country_x", "Country_y", "Country"], axis=1)
BRICS_table.columns=["Year", "BRA_Count", "RUS_Count", "IND_Count", "CHN_Count", "SA_Count"]

In [ ]:
BRICS_table.plot(kind = "line", x="Year", y=["BRA_Count", "RUS_Count", "IND_Count", "CHN_Count", "SA_Count"])
plt.title('Total Medals Over Time for BRICS', )
plt.xlabel('Year')
plt.ylabel('Medals')

##### US Medals in Relation to GDP

In [ ]:
#Creating Individual Graphs with GDP and Population

#US - GDP

x = list(demo_table["Year"][:13])
y1 = list(demo_table["USA_Total_Medals"][:13])
y2 = list(demo_table["US_GDP"][:13])
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Year')
ax1.set_ylabel('Medals', color='g')
ax2.set_ylabel('GDP (in tens of trillions)', color='b')
plt.title('Total US Medals in Relation to GDP')

plt.show()

##### US Medals in Relation to Population

In [ ]:
#US - Pop
x = list(demo_table["Year"][:13])
y1 = list(demo_table["USA_Total_Medals"][:13])
y2 = list(demo_table["US_Pop"][:13])
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Year')
ax1.set_ylabel('Medals', color='g')
ax2.set_ylabel('Population(in hundreds of millions)', color='b')
plt.title('Total US Medals in Relation to Population')

plt.show()

##### China Medals in Relation to GDP

In [ ]:
#China - GDP

x = list(demo_table["Year"][:13])
y1 = list(demo_table["China_Total_Medals"][:13])
y2 = list(demo_table["CHN_GDP"][:13])
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Year')
ax1.set_ylabel('Medals', color='g')
ax2.set_ylabel('GDP (in trillions)', color='b')
plt.title('Total China Medals in Relation to GDP')

plt.show()

##### China Medals in Relation to Population

In [ ]:
#China - Pop

x = list(demo_table["Year"][:13])
y1 = list(demo_table["China_Total_Medals"][:13])
y2 = list(demo_table["CHN_Pop"][:13])
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Year')
ax1.set_ylabel('Medals', color='g')
ax2.set_ylabel('Population (in billions)', color='b')
plt.title('Total China Medals in Relation to Population')

plt.show()

##### UK Medals in Relation to GDP

In [ ]:
#UK - GDP

x = list(demo_table["Year"][:13])
y1 = list(demo_table["UK_Total_Medals"][:13])
y2 = list(demo_table["UK_GDP"][:13])
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Year')
ax1.set_ylabel('Medals', color='g')
ax2.set_ylabel('GDP (in trillions)', color='b')
plt.title('Total UK Medals in Relation to GDP')

plt.show()

##### UK Medals in Relation to Population

In [ ]:
#UK - Population

x = list(demo_table["Year"][:13])
y1 = list(demo_table["UK_Total_Medals"][:13])
y2 = list(demo_table["UK_Pop"][:13])
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Year')
ax1.set_ylabel('Medals', color='g')
ax2.set_ylabel('Population (in tens of millions)', color='b')
plt.title('Total UK Medals in Relation to Population')

plt.show()

##### Russia Medals in Relation to GDP

In [ ]:
#Russia - GDP

x = list(demo_table["Year"][:13])
y1 = list(demo_table["Russia_Total_Medals"][:13])
y2 = list(demo_table["RUS_GDP"][:13])
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Year')
ax1.set_ylabel('Medals', color='g')
ax2.set_ylabel('GDP (in trillions)', color='b')
plt.title('Total Russia Medals in Relation to GDP')

plt.show()

##### Russia Medals in Relation to Population

In [ ]:
x = list(demo_table["Year"][:13])
y1 = list(demo_table["Russia_Total_Medals"][:13])
y2 = list(demo_table["RUS_Pop"][:13])
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Year')
ax1.set_ylabel('Medals', color='g')
ax2.set_ylabel('Population (in hundreds of millions)', color='b')
plt.title('Total Russia Medals in Relation to Population')

plt.show()

### Step 4: Prediction
We want to predict the trend in medals for 3 countries - US, UK, and Russia, based on the Year, Population, and GDP

In [ ]:
import statsmodels.formula.api as sm

#### Prediction for US Medals 

In [ ]:
# Normalize/relabel the time variables
demo_table["Year_Norm"] = (pd.to_numeric(demo_table["Year"])-1896)/10
# Create the Constant variable
demo_table["Constant"] = [1]*len(demo_table)

In [ ]:
# Fit a function using the OLS, based on Year and Constant
model = sm.ols(formula = 'USA_Total_Medals ~ np.power(Year_Norm, 2) + Year_Norm + Constant', data = demo_table).fit()

model.summary()

In [ ]:
# Adding predictions in the dataframe
demo_table["Prediction"] = pd.to_numeric(model.fittedvalues).astype(int)
#demo_table.set_index(keys="Year", inplace=True)

In [ ]:
# Plotting the actual data and the prediction
demo_table[["USA_Total_Medals","Prediction"]].plot(style=['bo','r--'], label=["data", "prediction"])
plt.xlabel("Year")
plt.ylabel("Medals Won")
plt.title("US Medals Predictions")

In [ ]:
# Compare predictions and forecast medals for the future
past = {"Year_Norm":10.8, "Year":2004, "Constant":1}
print "This model predicts the US would win", int(model.predict(past)[0]), "medals in 2004"
past = {"Year_Norm":11.2, "Year":2008, "Constant":1}
print "This model predicts the US would win", int(model.predict(past)[0]), "medals in 2008"
future = {"Year_Norm":11.6, "Year":2012, "Constant":1}
print "This model predicts the US will win", int(model.predict(future)[0]), "medals in 2012"
future = {"Year_Norm":12, "Year":2016, "Constant":1}
print "This model predicts the US will win", int(model.predict(future)[0]), "medals in 2016"

In [ ]:
demo_table

##### Prediction for China Medals

In [ ]:
model = sm.ols(formula = 'China_Total_Medals ~ np.power(Year_Norm, 2) + Year_Norm + Constant', data = demo_table).fit()

model.summary()

In [ ]:
demo_table["Prediction"] = pd.to_numeric(model.fittedvalues).astype(int)
#demo_table.set_index(keys="Year", inplace=True)

In [ ]:
# Plotting the actual data and the prediction
demo_table[["China_Total_Medals","Prediction"]].plot(style=['bo','r--'], label=["data", "prediction"])
plt.xlabel("Year")
plt.ylabel("Medals Won")
plt.title("China Medals Predictions")

In [ ]:
# Compare predictions and forecast medals for the future
past = {"Year_Norm":10.8, "Year":2004, "Constant":1}
print "This model predicts the China would win", int(model.predict(past)[0]), "medals in 2004"
past = {"Year_Norm":11.2, "Year":2008, "Constant":1}
print "This model predicts the China would win", int(model.predict(past)[0]), "medals in 2008"
future = {"Year_Norm":11.6, "Year":2012, "Constant":1}
print "This model predicts the China will win", int(model.predict(future)[0]), "medals in 2012"
future = {"Year_Norm":12, "Year":2016, "Constant":1}
print "This model predicts the China will win", int(model.predict(future)[0]), "medals in 2016"

##### Prediction for UK Medals

In [ ]:
model = sm.ols(formula = 'UK_Total_Medals ~ np.power(Year_Norm, 2) + Year_Norm + Constant', data = demo_table).fit()

model.summary()

In [ ]:
demo_table["Prediction"] = pd.to_numeric(model.fittedvalues).astype(int)
#demo_table.set_index(keys="Year", inplace=True)


In [ ]:
# Plotting the actual data and the prediction
demo_table[["UK_Total_Medals","Prediction"]].plot(style=['bo','r--'], label=["data", "prediction"])
plt.xlabel("Year")
plt.ylabel("Medals Won")
plt.title("UK Medals Predictions")

In [ ]:
# Compare predictions and forecast medals for the future
past = {"Year_Norm":10.8, "Year":2004, "Constant":1}
print "This model predicts the UK would win", int(model.predict(past)[0]), "medals in 2004"
past = {"Year_Norm":11.2, "Year":2008, "Constant":1}
print "This model predicts the UK would win", int(model.predict(past)[0]), "medals in 2008"
future = {"Year_Norm":11.6, "Year":2012, "Constant":1}
print "This model predicts the UK will win", int(model.predict(future)[0]), "medals in 2012"
future = {"Year_Norm":12, "Year":2016, "Constant":1}
print "This model predicts the UK will win", int(model.predict(future)[0]), "medals in 2016"

##### Prediction for Russia Medals

In [ ]:
model = sm.ols(formula = 'Russia_Total_Medals ~ np.power(Year_Norm, 2) + Year_Norm + Constant', data = demo_table).fit()

model.summary()

In [ ]:
demo_table["Prediction"] = pd.to_numeric(model.fittedvalues).astype(int)
#demo_table.set_index(keys="Year", inplace=True)


In [ ]:
# Plotting the actual data and the prediction
demo_table[["Russia_Total_Medals","Prediction"]].plot(style=['bo','r--'], label=["data", "prediction"])
plt.xlabel("Year")
plt.ylabel("Medals Won")
plt.title("Russia Medals Predictions")

In [ ]:
# Compare predictions and forecast medals for the future
past = {"Year_Norm":10.8, "Year":2004, "Constant":1}
print "This model predicts the Russia would win", int(model.predict(past)[0]), "medals in 2004"
past = {"Year_Norm":11.2, "Year":2008, "Constant":1}
print "This model predicts the Russia would win", int(model.predict(past)[0]), "medals in 2008"
future = {"Year_Norm":11.6, "Year":2012, "Constant":1}
print "This model predicts the Russia will win", int(model.predict(future)[0]), "medals in 2012"
future = {"Year_Norm":12, "Year":2016, "Constant":1}
print "This model predicts the Russia will win", int(model.predict(future)[0]), "medals in 2016"